In [2]:

# coding: utf-8

# In[1]:


import os
import os.path
import sys
import math
import pandas as pd
import numpy as np
import datetime
import openpyxl # for writing to excel
import xlrd # for writing to excel

class FolderOper(object):

    def __init__(self, goDir):
        self.folderOfProg = goDir
        self.curFolder = "c:\\"
        
    def changeFolder(self):
        """
        @author : Murat BAYRAV
        Aim     : Directory operations
        Date    : September 1, 2018
        Usage   : 
        Project : Training...
        """
        self.curFolder = os.getcwd()
        # print("{0} klasörü içindesiniz.".format(self.curFolder))
        try:
            os.chdir(self.folderOfProg)
            # print("Program, {0} isimli klasör içindedir.".format(self.folderOfProg))
        except:
            print("{0} isimli klasör bulunmamaktadır !".format(self.folderOfProg))
            print("Program sonlandırılıyor...")
            sys.exit(-100)
            
class ReadFileAndKey(object):
    
    def __init__(self, folderIs, fileIs, xlsxOr):
        self.constFolder = folderIs
        self.constFile = fileIs
        self.xlsxOrCsv = xlsxOr

    def readOneFile(self):
        """
        @author : Murat BAYRAV
        Aim     : Directory operations
        Date    : September 1, 2018
        Usage   : 
        Project : Training...
        """        
        # Check whether files are exist or not...
        # The code below returns True if file exists...
        if not os.path.isfile(self.constFolder + "\\" + self.constFile):
            print(self.constFile + " isimli dosya, " + self.constFolder + " isimli klasörde mevcut değildir !")
            sys.exit(1)
        # Read file...
        try:
            if self.xlsxOrCsv == "xlsx":
                # Default sheetname is zero... Very first sheet...
                dfTempIs = pd.read_excel(self.constFolder + "\\" + self.constFile)
            elif self.xlsxOrCsv == "csv":
                # Code page 1254 Windows-Turkish...
                dfTempIs = pd.read_csv(self.constFolder + "\\" + self.constFile, sep = ';', encoding = 'cp1254') 
            else:
                print("Dosya formatı bilinmiyor !")
                print("Program sonlandırılıyor...")
                sys.exit(18)
        except:
            print(self.constFolder + "\\" + self.constFile + " isimli dosya, okunamıyor !")
            print("Program sonlandırılıyor...")
            sys.exit(5)
        return dfTempIs

    def sanityCheck(self, dfTempIs, byColsAre):
        """
        @author : Murat BAYRAV
        Aim     : Directory operations
        Date    : September 1, 2018
        Usage   : 
        Project : Training...
        """        
        # Check columns are available...
        try:
            dfTempIs.sort_values(byColsAre, inplace = True)
        except:
            print("Dataframe " + ", ".join(byColsAre) + " isimli kolonları içermiyor !")
            sys.exit(7)
        return dfTempIs    
    
class AlgoritmaIKFA(object):
    """
    @author : Murat BAYRAV
    Aim     : Directory operations
    Date    : September 1, 2018
    Usage   : 
    Project : Training...
    """
    def __init__(self, dfPast, dfPers):
        self.dfIKFA = dfPast
        self.dfPersonel = dfPers
    
    def offerPersonel(self):
        """
        @author : Murat BAYRAV
        Aim     : Directory operations
        Date    : September 1, 2018
        Usage   : 
        Project : Training...
        """
        self.dfPersonel.reset_index(inplace = True)
        personelList = self.dfPersonel['PERSONEL'].tolist()
        self.dfIKFA = self.dfIKFA[["MUHASEBECI_ADI", "PERSONEL"]]
        dfIKFAGrouped = self.dfIKFA.groupby(["MUHASEBECI_ADI", "PERSONEL"])
        dfGecmis = dfIKFAGrouped.size().reset_index().rename(columns = {'MUHASEBECI_ADI':'MUHASEBECI', 'PERSONEL':'PERSONEL', 0:'ADET'})
        dfGecmisPivoted = dfGecmis.pivot(index = 'MUHASEBECI', columns = 'PERSONEL', values = 'ADET')
        # There may be more than one PERSONEL to have been assigned...
        # Thus, who is the most assigned ?
        # This PERSONEL will be optimum, if She/He is in list named personelList...
        newCol = list()
        for index, row in dfGecmisPivoted.iterrows(): # Dataframe loop...
            oneSerie = row[index : ] # One row...
            dfProcess = oneSerie.to_frame()
            dfProcess.dropna(inplace = True)
            dfProcess.reset_index(inplace = True)
            # Sort by assigments in order to obtain max...
            dfProcess.sort_values(by = dfProcess.columns[1], axis = 0, ascending = False, inplace = True)
            dfProcess.reset_index(inplace = True)
            # Is this PERSONEL found the list named personelList ?
            isFound = False
            for inc in range(0, len(dfProcess.index)): # dfProcess.columns
                # if this PERSONEL is not found, move on to the next...
                for pers in personelList:
                    if pers == dfProcess.loc[inc, 'PERSONEL']:
                        isFound = True
                        break
                if isFound:
                    break
            if not isFound: # No one exists exactly...
                pers = " " # Assign blank...
            ### Important ###
            # if an empty string is assigned to the pers...
            # then, groupby mechanism does not count NaNs in size() !
            #################
            newCol.append(pers)
        dfGecmisPivoted['ONERILEN'] = newCol
        return dfGecmisPivoted  
    
class SomeFuncs(object):

    def insertColumn(self, idx, initVal, columnName, gelenDf):
        """
        @author : Murat BAYRAV
        Aim     : insert column to a specific position...
        Date    : August 7, 2018
        Usage   : 
        Project : Training...
        """
        # newColValues can be a list, a Series, an array or a scalar...
        newColValues = list()
        for inc in range(0, len(gelenDf)):
            newColValues.append(initVal)
        try:    
            gelenDf.insert(loc = idx, column = columnName, value = newColValues)
        except:
            print("{0} isimli dataFrame için kolon ekleme işlemi sırasında bir hata oluştu !".format(gelenDf))
            print("Program sonlandırılıyor !")
            sys.exit(112)
        return gelenDf

    def spaceAdd(self, comingStr, byteFinal, bOrAfter = True):
        emptyList = list()
        lenOfStr = len(comingStr)
        for i in range(0, lenOfStr):
            emptyList.append(comingStr[i])
        for i in range(1, byteFinal - lenOfStr):
            if bOrAfter:
                emptyList.append(' ')
            else:    
                emptyList.insert(0, ' ')
        comingStr = "".join(emptyList)
        return comingStr
    
class WritingXlsxFile(object):
    
    def __init__(self, dataOf, pathOf, XlsxFileOf, SheetOf, wrLog):
        self.dataFrame = dataOf
        self.path = pathOf
        self.excelFileName = XlsxFileOf
        self.excelSheetName = SheetOf
        self.multiIndex = True
        self.writeLog = wrLog
    
    """ Write to Excel file..."""                    
    def writeExcelFile(self):
        try:
            """
            print("Dataframe   : {0}".format(self.dataFrame))
            print("Path        : {0}".format(self.path))
            print("Excel Name  : {0}".format(self.excelFileName))
            print("Concat      : {0}".format(self.path + self.excelFileName))
            print("Sheet Name  : {0}".format(self.excelSheetName))
            print("Multi Index : {0}".format(self.multiIndex))
            """
            excelFile = pd.ExcelWriter(self.path + self.excelFileName, engine = 'xlsxwriter')
            self.dataFrame.to_excel(excelFile, sheet_name = self.excelSheetName, index = self.multiIndex)
            excelFile.save # Save !!!
            print(self.excelFileName + " isimli dosyaya yazma işlemi yapıldı...")
            if self.writeLog:
                pass
        except:
            print("Excel dosyaya yazma hatası oluştu !!!")
            print("Program sonlandırılıyor...")
            sys.exit(111)
        return 0    
    
class WritingTextFile(object):
    
    def __init__(self, pathOf = "d:\\"):
        self.path = pathOf
        self.filePointer = None
        # now.year...now.microsecond
        # Current date and time using strftime...
        self.nameOfFile = "Log_" + datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S") + ".txt"

    def fileCreate(self):
        try:
            # * used. So, flags = compulsory.
            self.filePointer = os.open(self.path + self.nameOfFile, flags = os.O_RDWR|os.O_CREAT)
        except:
            print("Dosya oluşturma hatası oluştu !!!")
            print("Program sonlandırılıyor!!!")
            sys.exit(99)

    def fileClose(self):
        try:
            os.close(self.filePointer)
        except:
            print("Dosya kapatma hatası oluştu !!!")
            print("Program sonlandırılıyor!!!")
            sys.exit(44)

    def fileOpen(self):
        try:
            self.filePointer = os.open(self.path + self.nameOfFile, os.O_RDWR)        
        except:
            print("Dosya açma hatası oluştu !!!")
            print("Program sonlandırılıyor!!!")
            sys.exit(66)

    def fileDelete(self):
        try:
            os.remove(self.path + self.nameOfFile)
        except:
            print("Dosya silme hatası oluştu !!!")
            print("Program sonlandırılıyor!!!")
            sys.exit(88)

    def filePrint(self, str_):
        # Is file accessable ?
        if os.access(self.path + self.nameOfFile, os.R_OK):
            try:
                ### Important !!! ###
                # before writing a string into the file, encode it first...
                # str is a reserved word !
                # str_ = str_ + '\n' (carriage return and linefeed)
                line = str.encode(str_)
                os.write(self.filePointer, line)
            except Exception as ex:
                template = "{0} tipli bir hata oluştu. Argüman : \n{1!r}"
                message = template.format(type(ex).__name__, ex.args)
                print (message)
                print("Dosyaya yazma hatası oluştu !!!")
                print("program sonlandırılıyor !!!")
                sys.exit(99)
        else:
            print("Dosyaya Erişilemiyor...")
            print("program sonlandırılıyor !!!")
            sys.exit(101)

    def fileExist(self):
        if os.path.isfile(self.path + self.nameOfFile) and os.access(self.path + self.nameOfFile, os.R_OK):
            # File exists and is readable..
            return True
        else:
            pass
            # Either file is missing or is not readable...
        return False

